# Calculating Statistical Significance for Hypothesis Tests in CRO Pod
<br>

### Last updated: 2023-06-01

<br>
<br>
This notebook connects to the GA API to return data with the AB test variant identifiers. Hypothesis tests are then run to calculate statistical significance between control and variant groups.

## Load and mount the necessary Google Drive stuff

In [ ]:
# see all packages installed
# pip freeze

In [8]:
import pandas as pd

from google.colab import auth
auth.authenticate_user()

# import gspread
# from google.auth import default
# creds, _ = default()

# gc = gspread.authorize(creds)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the GA data

In [2]:
# https://www.jcchouinard.com/google-analytics-api-using-python/

#Load Libraries
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import httplib2
 
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '/content/drive/MyDrive/REPLACE_AS_NEEDED.json'


#Create service credentials
#Rename your JSON key to client_secrets.json and save it to your working folder
# ML: I overwrote the client_secrets.json to be the file json I downloaded from console.cloud.google.com
credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
  
#Create a service object
http = credentials.authorize(httplib2.Http())
service = build('analyticsreporting', 'v4', http=http, discoveryServiceUrl=('https://analyticsreporting.googleapis.com/$discovery/rest'))

## What parameters should we be querying? 

In [3]:
# Select the GA parameters to grab
viewId = 'REPLACE_AS_NEEDED' # Don't change for CRO stuff - Optimize tests are only linked to No Filters
startDate = '2023-05-14'
endDate = '2023-05-29'

## Grab the data

In [9]:
def ga_to_dataframe(response):
    # create a dictionary to hold our data
    data_dict = {}
  
    # Extract Data
    for report in response.get('reports', []):
  
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
  
        for row in rows:
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])
  
            for header, dimension in zip(dimensionHeaders, dimensions):
                if header not in data_dict:
                    data_dict[header] = [dimension]
                else:
                    data_dict[header].append(dimension)
  
            for i, values in enumerate(dateRangeValues):
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    if metricHeader.get('name') not in data_dict:
                        data_dict[metricHeader.get('name')] = [value]
                    else:
                        data_dict[metricHeader.get('name')].append(value)
    
    # Convert the dictionary into a DataFrame
    df = pd.DataFrame(data_dict)
    return df


response = service.reports().batchGet(
    body={
        'reportRequests': [
            {
                'viewId': viewId, #Add View ID from GA
                'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
                'metrics': [{'expression': 'ga:itemRevenue'}, {'expression': 'ga:transactionRevenue'}, {'expression': 'ga:itemQuantity'}], 
                'dimensions': [{"name": "ga:experimentCombination"}, {"name": "ga:transactionId"}, {"name": "ga:experimentName"}], 
                'orderBys': [{"fieldName": "ga:transactionRevenue", "sortOrder": "ASCENDING"}], 
                'pageSize': 100000
            }]
    }
).execute()

df = ga_to_dataframe(response)

In [ ]:
df['ga:itemRevenue'] = df['ga:itemRevenue'].astype(float)
df['ga:transactionRevenue'] = df['ga:transactionRevenue'].astype(float)
df['ga:itemQuantity'] = df['ga:itemQuantity'].astype(int)

## Do the hypothesis testing

### What experiment are we testing for? Find the <i>Experiment ID with Variant</i> for the specific test.

In [11]:
# What experiment are we testing for? Replace with your experiment ID - the 0 and 1 indicate whether a user was in the control or variant
ID_original = 'REPLACE_AS_NEEDED:0'
ID_variant = 'REPLACE_AS_NEEDED:1' # Removal of optional selectors

In [12]:
# Filter for just the transactions that happened in the original 
original = df[(df['ga:experimentCombination'] == ID_original) & (df['ga:itemQuantity'] > 0)]
variant = df[(df['ga:experimentCombination'] == ID_variant) & (df['ga:itemQuantity'] > 0)]

print('length of original:', len(original))
print('length of variant:', len(variant))

length of original: 584
length of variant: 566


In [14]:
# take just the revenue values from each original and variant
original_test = original['ga:itemRevenue']
variant_test = variant['ga:itemRevenue']

In [19]:
# What are the means and variances? PRODUCT/ITEM REVENUE
original_mean = original['ga:itemRevenue'].mean()
variant_mean = variant['ga:itemRevenue'].mean()
difference = variant_mean - original_mean

original_var = original['ga:itemRevenue'].var()
variant_var = variant['ga:itemRevenue'].var()

original_sum = original['ga:itemRevenue'].sum()
variant_sum = variant['ga:itemRevenue'].sum()

print(f"Original Mean: ${original_mean:,.2f}")
print(f"Variant Mean: ${variant_mean:,.2f}")
print(f"Difference: ${difference:,.2f}")
print(f"\nOriginal Variance: ${original_var:,.2f}")
print(f"Variant Variance: ${variant_var:,.2f}")
print(f"\nOriginal Sum of Rev.: ${original_sum:,.2f}")
print(f"Variant Sum of Rev.: ${variant_sum:,.2f}")

Original Mean: $1,137.48
Variant Mean: $1,212.42
Difference: $74.94

Original Variance: $714,423.00
Variant Variance: $738,827.11

Original Sum of Rev.: $664,288.81
Variant Sum of Rev.: $686,231.63


In [16]:
# Do the Hypothesis Test
from scipy.stats import t
from scipy.stats import ttest_ind

# What are the t stat and p-values?
t_stat, pvalue = ttest_ind(original_test, variant_test, alternative = 'less')

print('t-statistics:', t_stat, '\np-value:', pvalue)

t-statistics: -1.4907199638083692 
p-value: 0.06815486702131768
